## Resources:  
* https://making.lyst.com/lightfm/docs/lightfm.html
* https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b
* https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/


In [32]:
import pandas as pd
import numpy as np
from time import time

from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

from scipy.sparse import csr_matrix


In [33]:
plays = pd.read_csv('datasets/user_artists.dat', sep='\t')
artists = pd.read_csv('datasets/artists.dat', sep='\t', usecols=['id','name'])

# Merge (fusionner) artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [34]:
#---------------------------------------------------------------------------------------------------
print(80*("_"))
print("\nPlays info:\n")
plays.info()
print(80*("_"))
print("\nap info:\n")
ap.info()
print(80*("_"))
print("\nartist info:\n")
artists.info()

________________________________________________________________________________

Plays info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92834 entries, 0 to 92833
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userID    92834 non-null  int64
 1   artistID  92834 non-null  int64
 2   weight    92834 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB
________________________________________________________________________________

ap info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92834 entries, 0 to 92833
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         92834 non-null  int64 
 1   name       92834 non-null  object
 2   userID     92834 non-null  int64 
 3   artistID   92834 non-null  int64 
 4   playCount  92834 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 4.2+ MB
___________________________________________________

## Normalisation des données  
  
On n'utilisera la ligne:  
ap.playCount= np.log(ap.playCount + 0.1) # + 0.1 pour éviter les '0' (log(1)= 0)  
  
Résultat:  

Normalisation avec échelle Logarithmique  
['Depeche Mode' 'David Bowie' 'The Beatles' **'New Order'** **'Duran Duran'** 'The Cure' **'Pet Shop Boys'** **Radiohead'** **'Erasure'** **'Björk'**]  
   
Normalisation sans échelle logarithmique  
['Depeche Mode' 'David Bowie' 'Daft Punk' 'Queen' 'The Beatles' 'Coldplay' 'Madonna' 'Muse' 'Lady Gaga' 'The Cure']  
  
**En gras les différences**

In [35]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner").sort_values(['playCount'], ascending=False)

# Preprocessing

# On teste ici une approche logarithmique pour éviter l'effet "lady gaga" recommandé pour tous 
ap.playCount= np.log(ap.playCount + 0.1) # + 0.1 pour éviter les '0' (log(1)= 0)

pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
#print(ap)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values

# Show sparsity . C'est plutôt une densité.... Indique le pourcentage de valeur non nul de la matrice. 
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print(f"sparsity: {sparsity:.2f} %")


sparsity: 0.28 %


In [36]:
#ratings.shape
#ratings

In [37]:
print(ap.info())
ap.artistID

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92834 entries, 2800 to 63982
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               92834 non-null  int64  
 1   name             92834 non-null  object 
 2   userID           92834 non-null  int64  
 3   artistID         92834 non-null  int64  
 4   playCount        92834 non-null  float64
 5   totalUsers       92834 non-null  int64  
 6   totalPlays       92834 non-null  int64  
 7   avgPlays         92834 non-null  float64
 8   playCountScaled  92834 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 7.1+ MB
None


2800        72
35843      792
27302      511
8152       203
26670      498
         ...  
38688      913
32955      697
71811     4988
91319    17080
63982     3201
Name: artistID, Length: 92834, dtype: int64

In [38]:
# Build a sparse matrix                      PEUT ON créer une matrice creuse coo directement ?
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


In [39]:
# Build data references + train test
Xcoo = X.tocoo()
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

In [40]:
# Train
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10, num_threads=2)

In [41]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.39, test 0.13.
AUC: train 0.97, test 0.86.


In [42]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items[:10])

['The Beatles' 'Depeche Mode' 'Pet Shop Boys' 'Madonna' 'a-ha' 'Coldplay'
 'Michael Jackson' 'Duran Duran' 'New Order' 'Queen']


In [43]:
#test.shape
#train.shape


In [44]:
param_loss= ["warp", "bpr", "warp-kos", "logistic"]
resultats= ""
dictionnaire_param_loss= {}
for ploss in param_loss:
    tps_deb= time()

    # Train
    model = LightFM(learning_rate=0.05, loss= ploss)
    model.fit(train, epochs=10, num_threads=2)

    # Evaluate
    train_precision = precision_at_k(model, train, k=10).mean()
    test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

    train_auc = auc_score(model, train).mean()
    test_auc = auc_score(model, test, train_interactions=train).mean()
    
    # Predict
    scores = model.predict(0, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    
    tps_fin= time()


    ch= "Méthode "+ ploss + ":"+ f"\nPrecision: train {train_precision:.2f}, test {test_precision:.2f}." + \
    f"\nAUC: train {train_auc:.2f}, test {test_auc:.2f}." +\
    f"\nRecommandation:\n{top_items}" + "\n\n"
    resultats+= ch
    dictionnaire_param_loss["Méthode " + ploss]={"Précision train": train_precision, 
                                                "Précision test": test_precision, "AUC train": train_auc, 
                                                "AUC test": test_auc, "Temps": tps_fin-tps_deb,
                                                "Recommandation": top_items}

print(resultats)

Méthode warp:
Precision: train 0.38, test 0.13.
AUC: train 0.96, test 0.85.
Recommandation:
['Madonna' 'Michael Jackson' 'Lady Gaga' ... 'Texas in July' 'that dog.'
 'Pipedown']

Méthode bpr:
Precision: train 0.36, test 0.12.
AUC: train 0.85, test 0.78.
Recommandation:
['Depeche Mode' 'Madonna' 'New Order' ... 'Miley Cyrus' 'Ke$ha'
 'Linkin Park']

Méthode warp-kos:
Precision: train 0.34, test 0.12.
AUC: train 0.89, test 0.82.
Recommandation:
['David Bowie' 'The Beatles' 'Björk' ... 'Nyze' 'Berlins Most Wanted'
 'Hostage Calm']

Méthode logistic:
Precision: train 0.20, test 0.07.
AUC: train 0.89, test 0.81.
Recommandation:
['Lady Gaga' 'Britney Spears' 'Rihanna' ... 'Edan' 'Estudio Base'
 'Pleq & Chihiro']




### Le paramètre warp donne le meilleur résultat avec un écart important. Il sera donc choisi et fixer. Les autres paramètres seront optimisés en utilisant GridSearchCV.

Ressource: 
cours factorisation matrice + grid search CV
    https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/

In [45]:
dictionnaire_param_loss

{'Méthode warp': {'Précision train': 0.37582183,
  'Précision test': 0.1298077,
  'AUC train': 0.9628738,
  'AUC test': 0.8549985,
  'Temps': 9.884975671768188,
  'Recommandation': array(['Madonna', 'Michael Jackson', 'Lady Gaga', ..., 'Texas in July',
         'that dog.', 'Pipedown'], dtype=object)},
 'Méthode bpr': {'Précision train': 0.36436903,
  'Précision test': 0.121634625,
  'AUC train': 0.85374516,
  'AUC test': 0.7810532,
  'Temps': 10.457010746002197,
  'Recommandation': array(['Depeche Mode', 'Madonna', 'New Order', ..., 'Miley Cyrus',
         'Ke$ha', 'Linkin Park'], dtype=object)},
 'Méthode warp-kos': {'Précision train': 0.34008482,
  'Précision test': 0.12179488,
  'AUC train': 0.8870051,
  'AUC test': 0.8193632,
  'Temps': 10.963048934936523,
  'Recommandation': array(['David Bowie', 'The Beatles', 'Björk', ..., 'Nyze',
         'Berlins Most Wanted', 'Hostage Calm'], dtype=object)},
 'Méthode logistic': {'Précision train': 0.19718982,
  'Précision test': 0.06917736,

In [49]:
#param_learning_rate= np.arange(0.01, 0.2, 0.02)
#param_epoch= np.arange(5,100,5)
param_learning_rate= [0.01, 0.05, 0.1]
param_epoch= [5, 10, 15]
ploss= "warp"

#print(param_learning_rate)
#print(param_epoch)

resultats= ""
dictionnaire_learningrate_epoch= {}

for learning_rate in [0.01, 0.05, 0.1]:
    for epoch in param_epoch:
        
        tps_deb= time()

        # Train
        model = LightFM(learning_rate= learning_rate, loss= ploss)
        model.fit(train, epochs= epoch, num_threads=2)

        # Evaluate
        train_precision = precision_at_k(model, train, k=10).mean()
        test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

        train_auc = auc_score(model, train).mean()
        test_auc = auc_score(model, test, train_interactions=train).mean()

        # Predict
        scores = model.predict(0, np.arange(n_items))
        top_items = artist_names[np.argsort(-scores)]

        tps_fin= time()


        ch= "Méthode "+ ploss + ":"+ f"\nPrecision: train {train_precision:.2f}, test {test_precision:.2f}." + \
        f"\nAUC: train {train_auc:.2f}, test {test_auc:.2f}." +\
        f"\nRecommandation:\n{top_items[:10]}" + "\n\n"
        resultats+= ch
        dictionnaire_learningrate_epoch["learning_rate" + str(learning_rate) + " epoch " + str(epoch)]= \
            {"Précision train": train_precision, "Précision test": test_precision, "AUC train": train_auc, \
             "AUC test": test_auc, "Temps": tps_fin-tps_deb, "Recommandation": top_items[:10]}


In [50]:
dictionnaire_learningrate_epoch


{'learning_rate0.01 epoch 5': {'Précision train': 0.25784728,
  'Précision test': 0.090384625,
  'AUC train': 0.8703303,
  'AUC test': 0.8024738,
  'Temps': 10.245213031768799,
  'Recommandation': array(['Lady Gaga', 'Britney Spears', 'Rihanna', 'Katy Perry', 'Madonna',
         'Paramore', 'The Beatles', 'Avril Lavigne', 'Christina Aguilera',
         'Beyoncé'], dtype=object)},
 'learning_rate0.01 epoch 10': {'Précision train': 0.28054082,
  'Précision test': 0.098023504,
  'AUC train': 0.887231,
  'AUC test': 0.80903226,
  'Temps': 10.064653158187866,
  'Recommandation': array(['Lady Gaga', 'Britney Spears', 'The Beatles', 'Rihanna', 'Muse',
         'Katy Perry', 'Madonna', 'Christina Aguilera', 'Paramore',
         'Avril Lavigne'], dtype=object)},
 'learning_rate0.01 epoch 15': {'Précision train': 0.2828738,
  'Précision test': 0.09476496,
  'AUC train': 0.8960393,
  'AUC test': 0.81631577,
  'Temps': 10.277898788452148,
  'Recommandation': array(['Lady Gaga', 'Britney Spears', '

In [51]:
print(resultats)

Méthode warp:
Precision: train 0.26, test 0.09.
AUC: train 0.87, test 0.80.
Recommandation:
['Lady Gaga' 'Britney Spears' 'Rihanna' 'Katy Perry' 'Madonna' 'Paramore'
 'The Beatles' 'Avril Lavigne' 'Christina Aguilera' 'Beyoncé']

Méthode warp:
Precision: train 0.28, test 0.10.
AUC: train 0.89, test 0.81.
Recommandation:
['Lady Gaga' 'Britney Spears' 'The Beatles' 'Rihanna' 'Muse' 'Katy Perry'
 'Madonna' 'Christina Aguilera' 'Paramore' 'Avril Lavigne']

Méthode warp:
Precision: train 0.28, test 0.09.
AUC: train 0.90, test 0.82.
Recommandation:
['Lady Gaga' 'Britney Spears' 'Madonna' 'Rihanna' 'The Beatles' 'Beyoncé'
 'Christina Aguilera' 'Katy Perry' 'Avril Lavigne' 'Muse']

Méthode warp:
Precision: train 0.35, test 0.12.
AUC: train 0.94, test 0.84.
Recommandation:
['Muse' 'Depeche Mode' 'The Beatles' 'Coldplay' 'The Cure' 'The Killers'
 'Radiohead' 'Green Day' 'Arctic Monkeys' 'Madonna']

Méthode warp:
Precision: train 0.39, test 0.14.
AUC: train 0.97, test 0.86.
Recommandation:
['Mado


# Recommander Systems

Construire, comprendre et tuner un système de recommandation.

# Description

## Familarisation

Les systèmes de recommandations sont utilisé traditionnellement et comme le nom l'indique pour recommander du contenu à des utilisateurs.
Par exemple pour recommander un film à des utilisateurs en fonctions de ceux qu'ils ont vue, ou de la musique, ou des vidéos ou encore implémenter des fonctionnalités "more like this".

Nous allons commencer par suivre et reproduire les étapes de ce tuto: 

*  https://www.datacamp.com/community/tutorials/recommender-systems-python

En assumant que vous avez peu de RAM, nous allons nous arrêter au moment de calculer la  `compute_sim` variable.


**step1 : simple recommander**
Quelle est la complexité en mémoire de cette opération ?
(utiliser cosine_similarity qui utilise moins de mémoire (quand même 8Go, possible sur collab)
Cela rentre t'il sur votre machine ?

Qu'essaye de faire l'auteur avec ce calcul ?
Comment pouvons-nous contourner ce problème ?


**step2 : content based recommander**

implémenter la deuxiéme partie en évitant le produit de matrice.

**step3 : amélioration**

coder les 2 améliorations :
1. Introduce a popularity filter: this recommender would take the 30 most similar movies, calculate the weighted ratings (using the IMDB formula from above), sort movies based on this rating, and return the top 10 movies.
2. Use the PCA to improve the speed of your similarity search with 100 components. Does the result are coherent.


## LastFM Project

M. Pontier vous contact pour l'aider à construire un système de recommandation. Il dispose d'une base de données comportant des données concernant ses utilisateurs (anonymisé) contenant les artistes qu'ils écoutent sur sa plateforme ainsi que le nombre d'écoutes. Monsieur pontier souhaite recommander à ses utilisateur  des artistes qu'il n'ont pas encore écoutés, et cela en fonction de leurs préférences musicale.

Monsieur pontier souhaite utiliser la librairie Lightfm, avec laquelle il a déjà un driver permettant de charger ses données qu'il vous fournit, un vrai bonus.
Monsieur Pontier à pu voir que la documentation comporte plusieurs modèle, il souhaite évaluer les modèle sur une jeux de train/test et utiliser le meilleurs modéle.

Pour l'évaluation, il souhaite comparer la mesure AUC, la précision et le rappel (visiter la documentation de Lightfm), qui devront être présenté dans un tableau.


#### Bonus 1

Comparer les résulats de l'AUC avec le meilleurs modéle de lightfm et une PCA (TruncatedSDV).


#### Bonus 2

L'apprentissage devant être le plus rapide possible tout en obtenant les meilleurs résultats, il vous est demandé de trouver le nombre d'itération permettant d'atteindre la convergence de 95% de la valeur maximal d'AUC sur le jeux de test.


### Veille

Quelle système de recommandation allez vous mettre en place ?

Qu'est ce que Lightfm ?

Qu'est ce un système de recommandation dit à "implicit feedback" ? Et a "explicit feedback ?


### Ressources: 

* LightFM: https://github.com/lyst/lightfm
* Jeux de données Last.fm : https://grouplens.org/datasets/hetrec-2011/
* https://towardsdatascience.com/recommendation-system-in-python-lightfm-61c85010ce17
  
  

# Bout de code ...

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [23]:
data1= np.array([["toto","titi","tutu","tyty","tata","tete"],[10,11,12,13,14,15]]).T
df1= pd.DataFrame(data1, columns=["Artiste","id"])
print(df1)
data2= np.array([["d2toto","d2titi","d2tutu","d2tyty","d2tata","d2tete"],[10,11,12,13,14,15]]).T
df2= pd.DataFrame(data2, columns=["Utilisateur","id"])
print(df2)

  Artiste  id
0    toto  10
1    titi  11
2    tutu  12
3    tyty  13
4    tata  14
5    tete  15
  Utilisateur  id
0      d2toto  10
1      d2titi  11
2      d2tutu  12
3      d2tyty  13
4      d2tata  14
5      d2tete  15


In [24]:
df1df2 = pd.merge(df1, df2, how="inner")
df1df2

,Artiste,id,Utilisateur
0,toto,10,d2toto
1,titi,11,d2titi
2,tutu,12,d2tutu
3,tyty,13,d2tyty
4,tata,14,d2tata
5,tete,15,d2tete
